# Cohort # 1 MLOPs-Zoomcap 2023

Load of libraries and modules needed

In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

Download 2022 january and february .PARQUET files

In [3]:
!wget -N -v --no-check-certificate https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!wget -N -v --no-check-certificate https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2023-05-19 21:05:01--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolviendo d37ci6vzurychx.cloudfront.net... 65.8.245.50, 65.8.245.51, 65.8.245.171, ...
Connecting to d37ci6vzurychx.cloudfront.net|65.8.245.50|:443... conectado.
  Unable to locally verify the issuer's authority.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: `yellow_tripdata_2022-01.parquet'

     0K .......... .......... .......... .......... ..........  0%  408K 91s
    50K .......... .......... .......... .......... ..........  0%  612K 76s
   100K .......... .......... .......... .......... ..........  0%  965K 63s
   150K .......... .......... .......... .......... ..........  0%  471K 67s
   200K .......... .......... .......... .......... ..........  0%  866K 62s
   250K .......... .......... ......

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2023-05-19 21:05:31--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolviendo d37ci6vzurychx.cloudfront.net... 65.8.245.50, 65.8.245.51, 65.8.245.171, ...
Connecting to d37ci6vzurychx.cloudfront.net|65.8.245.50|:443... conectado.
  Unable to locally verify the issuer's authority.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 45616512 (44M) [application/x-www-form-urlencoded]
Saving to: `yellow_tripdata_2022-02.parquet'

     0K .......... .......... .......... .......... ..........  0%  486K 91s
    50K .......... .......... .......... .......... ..........  0%  571K 85s
   100K .......... .......... .......... .......... ..........  0% 1009K 71s
   150K .......... .......... .......... .......... ..........  0%  590K 72s
   200K .......... .......... .......... .......... ..........  0% 1.71M 63s
   250K .......... .......... ......

### Q1
**Read the data for January. How many columns are there?**

Loading january data from .PARQUET file

In [4]:
df_train = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
df_train.shape

(2463931, 19)

There are 19 columns in the dataframe

Computing duration in minutes

In [6]:
# df_train['duration'] equals diference between pickup and dropoff in seconds
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
# Convert df_train['duration'] to minutes
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds()/60)

### Q2
**What's the standard deviation of the trips duration in January?**

In [7]:
df_train['duration'].describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

The standar deviation is 46.445 minutes

Dropping outliers

In [8]:
# filter only duration over 1 minutes and less than 1 hour
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

### Q3
**What fraction of the records left after you dropped the outliers?**

In [9]:
df_train.shape

(2421440, 20)

In [10]:
round(2421440/2463931*100,2)

98.28

One hot encoding

In [11]:
# select the features to train the model
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
# convert the datatype of categorical features from integer to string
df_train[categorical] = df_train[categorical].astype(str)

In [12]:
# convert the dataframe into a dictionary
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [13]:
# create a DictVectorizer and transform the lists of feature-value mappings to vectors
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

### Q4
**What's the dimensionality of this matrix (number of columns)?**

In [14]:
X_train.shape

(2421440, 516)

Training a model

In [15]:
# select the target feature for train the model
target = 'duration'
y_train = df_train[target].values

In [16]:
# define the model as LinearRegression class
lr = LinearRegression()
# train the model
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
# test prediction with X_train data
y_pred = lr.predict(X_train)

### Q5
**What's the RMSE on train?**

In [18]:
# mean squared value between predicted values and y_pred values
round(mean_squared_error(y_train, y_pred, squared=False),2)

7.0

Evaluating the model

In [19]:
df_val = pd.read_parquet('yellow_tripdata_2022-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds()/60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

y_val = df_val[target].values

In [20]:
y_pred = lr.predict(X_val)

### Q6
**What's the RMSE on validation?**

In [21]:
round(mean_squared_error(y_val, lr.predict(X_val), squared=False),3)

7.795